In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-09-26 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-26 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-26 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-26 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-26 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["timeline_index", "speaker"]
standardise_method = [None, None]
include_features_in_path = True
include_features_in_input = False

In [9]:
num_epochs = 100
dimensions = [15] # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

# w=5

In [10]:
size = 5

## UMAP

In [11]:
swmhau_network_umap_kfold_5, best_swmhau_network_umap_kfold_5, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_5_kfold_best_model.csv


In [12]:
swmhau_network_umap_kfold_5.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "learning_rate"
]).mean()

/tmp/ipykernel_1503371/539912721.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_5.groupby([


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.640336   
                                                                         0.0005         0.660197   
                                                                         0.0010         0.658467   
                                                          (512, 512)     0.0001         0.643346   
                                                                         0.0005         0.659954   
                                                                         0.0010         0.652723   
                                     12        1          (256, 256)     0.0001         0.640648   
                                                                         0.0005         0.662030   
                                                                         0.0010         0.657844   
                                                          (512, 512)     0.0001         0.649609   
                                                                         0.0005         0.660992   
                                                                         0.0010         0.652273   
           12              3         10        1          (256, 256)     0.0001         0.643070   
                                                                         0.0005         0.659262   
                                                                         0.0010         0.658951   
                                                          (512, 512)     0.0001         0.641236   
                                                                         0.0005         0.663726   
                                                                         0.0010         0.656771   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.482679   
                                                                         0.0005         0.550703   
                                                                         0.0010         0.547660   
                                                          (512, 512)     0.0001         0.506706   
                                                                         0.0005         0.549407   
                                                                         0.0010         0.541600   
                                     12        1          (256, 256)     0.0001         0.473925   
                                                                         0.0005         0.547727   
                                                                         0.0010         0.548781   
                                                          (512, 512)     0.0001         0.516010   
                                                                         0.0005         0.548750   
                                                                         0.0010         0.543569   
           12              3         10        1          (256, 256)     0.0001         0.485430   
                                                                         0.0005         0.548278   
                                                                         0.0010         0.549760   
                                                          (512, 512)     0.0001         0.503383   
                                                                         0.0005         0.550334   
                                                                         0.0010         0.546721   

                                                                                      

In [13]:
best_swmhau_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.655595,0.548930,"[0.7752996005326231, 0.5074246924056003, 0.364...",0.541514,"[0.8008940852819807, 0.5072094995759118, 0.316...",0.562501,"[0.7512903225806452, 0.5076400679117148, 0.428...",None,0.650943,...,0.001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.655387,0.547256,"[0.775890231950343, 0.4795321637426901, 0.3863...",0.542650,"[0.78590337524818, 0.5100478468899522, 0.332]",0.560186,"[0.7661290322580645, 0.4524617996604414, 0.461...",None,0.652516,...,0.001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.661823,0.550473,"[0.7834549878345499, 0.4817044928207504, 0.386...",0.548715,"[0.7879282218597063, 0.5300713557594292, 0.328...",0.563282,"[0.7790322580645161, 0.44142614601018676, 0.46...",None,0.660849,...,0.001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_umap_kfold_5["f1"].mean()

0.5488864344199431

In [15]:
best_swmhau_network_umap_kfold_5["precision"].mean()

0.5442931118519999

In [16]:
best_swmhau_network_umap_kfold_5["recall"].mean()

0.5619894905536428

In [17]:
np.stack(best_swmhau_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.77821494, 0.48955378, 0.37889058])

In [18]:
np.stack(best_swmhau_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.79157523, 0.51577623, 0.32552787])

In [19]:
np.stack(best_swmhau_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.76548387, 0.467176  , 0.4533086 ])

## Random Projections

In [20]:
swmhau_network_grp_kfold_5, best_swmhau_network_grp_kfold_5, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_5_kfold_best_model.csv


In [21]:
swmhau_network_grp_kfold_5.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "dropout_rate",
    "learning_rate"
]).mean()

/tmp/ipykernel_1503371/2125086320.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_5.groupby([


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.651581   
                                                                                      0.0005         0.665421   
                                                                                      0.0010         0.653034   
                                                                         0.2          0.0001         0.652066   
                                                                                      0.0005         0.664591   
                                                                                      0.0010         0.660577   
                                                          (512, 512)     0.1          0.0001         0.663553   
                                                                                      0.0005         0.653934   
                                                                                      0.0010         0.651235   
                                                                         0.2          0.0001         0.664037   
                                                                                      0.0005         0.661131   
                                                                                      0.0010         0.656979   
                                     12        1          (256, 256)     0.1          0.0001         0.649021   
                                                                                      0.0005         0.659262   
                                                                                      0.0010         0.654349   
                                                                         0.2          0.0001         0.656218   
                                                                                      0.0005         0.663691   
                                                                                      0.0010         0.657048   
                                                          (512, 512)     0.1          0.0001         0.662169   
                                                                                      0.0005         0.655318   
                                                                                      0.0010         0.655664   
                                                                         0.2          0.0001         0.660231   
                                                                                      0.0005         0.661961   
                                                                                      0.0010         0.655733   
           12              3         10        1          (256, 256)     0.1          0.0001         0.659470   
                                                                                      0.0005         0.660439   
                                                                                      0.0010         0.652342   
                                                                         0.2          0.0001         0.658086   
                                                                                      0.0005         0.662930   
                                                                                      0.0010         0.654349   
                                                          (512, 512)     0.1          0.0001         0.662446   
                                                                                      0.0005         0.659608   
                                                                                      0.0010         0.657256   
                                                                         0.2          0.0001         0.659539   
                                 

In [22]:
best_swmhau_network_grp_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.665767,0.553378,"[0.7845232313420755, 0.4928057553956835, 0.382...",0.550471,"[0.7874553136171596, 0.5239005736137667, 0.340...",0.561552,"[0.7816129032258065, 0.46519524617996605, 0.43...",None,0.658491,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.658086,0.548396,"[0.7786657967705106, 0.48490310950878773, 0.38...",0.544400,"[0.7875288683602771, 0.5168107588856868, 0.328...",0.560417,"[0.77, 0.4567062818336163, 0.45454545454545453]",None,0.656447,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.659539,0.553164,"[0.7791270101739416, 0.48851590106007065, 0.39...",0.547113,"[0.7929191716766867, 0.509208103130755, 0.3392...",0.566356,"[0.7658064516129032, 0.4694397283531409, 0.463...",None,0.650786,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [23]:
best_swmhau_network_grp_kfold_5["f1"].mean()

0.5516462706171555

In [24]:
best_swmhau_network_grp_kfold_5["precision"].mean()

0.5473280569283129

In [25]:
best_swmhau_network_grp_kfold_5["recall"].mean()

0.5627750916181671

In [26]:
np.stack(best_swmhau_network_grp_kfold_5["f1_scores"]).mean(axis=0)

array([0.78077201, 0.48874159, 0.38542521])

In [27]:
np.stack(best_swmhau_network_grp_kfold_5["precision_scores"]).mean(axis=0)

array([0.78930112, 0.51663981, 0.33604324])

In [28]:
np.stack(best_swmhau_network_grp_kfold_5["recall_scores"]).mean(axis=0)

array([0.77247312, 0.46378042, 0.45207174])

# w=11

In [10]:
size = 11

## UMAP

In [30]:
swmhau_network_umap_kfold_11, best_swmhau_network_umap_kfold_11, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_11_kfold_best_model.csv


In [31]:
swmhau_network_umap_kfold_11.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "learning_rate"
]).mean()

/tmp/ipykernel_1503371/2974833079.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_11.groupby([


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.632275   
                                                                         0.0005         0.662653   
                                                                         0.0010         0.660716   
                                                          (512, 512)     0.0001         0.640648   
                                                                         0.0005         0.663103   
                                                                         0.0010         0.660958   
                                     12        1          (256, 256)     0.0001         0.633347   
                                                                         0.0005         0.667324   
                                                                         0.0010         0.662619   
                                                          (512, 512)     0.0001         0.638399   
                                                                         0.0005         0.664210   
                                                                         0.0010         0.661442   
           12              3         10        1          (256, 256)     0.0001         0.642447   
                                                                         0.0005         0.658778   
                                                                         0.0010         0.662065   
                                                          (512, 512)     0.0001         0.638330   
                                                                         0.0005         0.656529   
                                                                         0.0010         0.661131   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.441281   
                                                                         0.0005         0.552123   
                                                                         0.0010         0.552117   
                                                          (512, 512)     0.0001         0.491061   
                                                                         0.0005         0.552675   
                                                                         0.0010         0.550473   
                                     12        1          (256, 256)     0.0001         0.429898   
                                                                         0.0005         0.555524   
                                                                         0.0010         0.552202   
                                                          (512, 512)     0.0001         0.489445   
                                                                         0.0005         0.551092   
                                                                         0.0010         0.548946   
           12              3         10        1          (256, 256)     0.0001         0.476197   
                                                                         0.0005         0.547315   
                                                                         0.0010         0.554947   
                                                          (512, 512)     0.0001         0.491608   
                                                                         0.0005         0.543501   
                                                                         0.0010         0.550499   

                                                                                      

In [32]:
best_swmhau_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.654349,0.555269,"[0.7724321624771101, 0.4979919678714859, 0.395...",0.548833,"[0.7980736154110767, 0.5249294449670743, 0.323...",0.576807,"[0.7483870967741936, 0.47368421052631576, 0.50...",None,0.647799,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.672203,0.555992,"[0.7885016862052353, 0.5013309671694764, 0.378...",0.555080,"[0.7850975375759514, 0.525092936802974, 0.3550...",0.558672,"[0.7919354838709678, 0.4796264855687606, 0.404...",None,0.667925,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.659539,0.550721,"[0.7789542270728131, 0.4851485148514852, 0.388...",0.546523,"[0.786771964461994, 0.5162835249042146, 0.3365...",0.562367,"[0.7712903225806451, 0.45755517826825126, 0.45...",None,0.657233,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [33]:
best_swmhau_network_umap_kfold_11["f1"].mean()

0.553993954974317

In [34]:
best_swmhau_network_umap_kfold_11["precision"].mean()

0.5501449814188412

In [35]:
best_swmhau_network_umap_kfold_11["recall"].mean()

0.5659484768337545

In [36]:
np.stack(best_swmhau_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.77996269, 0.49482382, 0.38719536])

In [37]:
np.stack(best_swmhau_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.78998104, 0.52210197, 0.33835194])

In [38]:
np.stack(best_swmhau_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.77053763, 0.47028862, 0.45701917])

## Random Projections

In [11]:
swmhau_network_grp_kfold_11, best_swmhau_network_grp_kfold_11, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_11_kfold_best_model.csv


In [12]:
swmhau_network_grp_kfold_11.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "dropout_rate",
    "learning_rate"
]).mean()

/tmp/ipykernel_3899100/12671380.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_11.groupby([


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.655387   
                                                                                      0.0005         0.668189   
                                                                                      0.0010         0.655456   
                                                                         0.2          0.0001         0.655110   
                                                                                      0.0005         0.665905   
                                                                                      0.0010         0.655526   
                                                          (512, 512)     0.1          0.0001         0.656910   
                                                                                      0.0005         0.662238   
                                                                                      0.0010         0.648675   
                                                                         0.2          0.0001         0.657532   
                                                                                      0.0005         0.666874   
                                                                                      0.0010         0.655941   
                                     12        1          (256, 256)     0.1          0.0001         0.660785   
                                                                                      0.0005         0.658017   
                                                                                      0.0010         0.651858   
                                                                         0.2          0.0001         0.660923   
                                                                                      0.0005         0.668120   
                                                                                      0.0010         0.658017   
                                                          (512, 512)     0.1          0.0001         0.660370   
                                                                                      0.0005         0.663068   
                                                                                      0.0010         0.654211   
                                                                         0.2          0.0001         0.661062   
                                                                                      0.0005         0.666736   
                                                                                      0.0010         0.656079   
           12              3         10        1          (256, 256)     0.1          0.0001         0.655941   
                                                                                      0.0005         0.656287   
                                                                                      0.0010         0.652827   
                                                                         0.2          0.0001         0.655387   
                                                                                      0.0005         0.662653   
                                                                                      0.0010         0.659885   
                                                          (512, 512)     0.1          0.0001         0.664383   
                                                                                      0.0005         0.660646   
                                                                                      0.0010         0.657117   
                                                                         0.2          0.0001         0.662722   
                                 

In [13]:
best_swmhau_network_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.675109,0.561004,"[0.7927179814755669, 0.48633626679018066, 0.40...",0.561205,"[0.7849462365591398, 0.5351681957186545, 0.363...",0.566954,"[0.8006451612903226, 0.4456706281833616, 0.454...",None,0.661950,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.664106,0.557450,"[0.7810709022433274, 0.4993441189331001, 0.391...",0.551558,"[0.7931493182573994, 0.5148782687105501, 0.346...",0.568303,"[0.7693548387096775, 0.4847198641765705, 0.450...",None,0.663208,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.665144,0.555677,"[0.7845953002610966, 0.493368700265252, 0.3890...",0.550649,"[0.7939233817701453, 0.514760147601476, 0.3432...",0.566049,"[0.775483870967742, 0.47368421052631576, 0.448...",None,0.655660,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_grp_kfold_11["f1"].mean()

0.5580437121150238

In [15]:
best_swmhau_network_grp_kfold_11["precision"].mean()

0.5544707877626771

In [16]:
best_swmhau_network_grp_kfold_11["recall"].mean()

0.5671020555158319

In [17]:
np.stack(best_swmhau_network_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.78612806, 0.49301636, 0.39498671])

In [18]:
np.stack(best_swmhau_network_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.79067298, 0.5216022 , 0.35113718])

In [19]:
np.stack(best_swmhau_network_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.78182796, 0.4680249 , 0.45145331])

# w=20

In [10]:
size = 20

## UMAP

In [11]:
swmhau_network_umap_kfold_20, best_swmhau_network_umap_kfold_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold_best_model.csv


In [12]:
swmhau_network_umap_kfold_20.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "learning_rate"
]).mean()

/tmp/ipykernel_2642323/2207682611.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_20.groupby([


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.628884   
                                                                         0.0005         0.666874   
                                                                         0.0010         0.659954   
                                                          (512, 512)     0.0001         0.637811   
                                                                         0.0005         0.664521   
                                                                         0.0010         0.661165   
                                     12        1          (256, 256)     0.0001         0.628434   
                                                                         0.0005         0.662480   
                                                                         0.0010         0.664175   
                                                          (512, 512)     0.0001         0.637499   
                                                                         0.0005         0.665352   
                                                                         0.0010         0.662376   
           12              3         10        1          (256, 256)     0.0001         0.640890   
                                                                         0.0005         0.661408   
                                                                         0.0010         0.662203   
                                                          (512, 512)     0.0001         0.646011   
                                                                         0.0005         0.666632   
                                                                         0.0010         0.663760   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.418432   
                                                                         0.0005         0.554198   
                                                                         0.0010         0.549599   
                                                          (512, 512)     0.0001         0.476903   
                                                                         0.0005         0.552670   
                                                                         0.0010         0.552310   
                                     12        1          (256, 256)     0.0001         0.424375   
                                                                         0.0005         0.554140   
                                                                         0.0010         0.553284   
                                                          (512, 512)     0.0001         0.469893   
                                                                         0.0005         0.554957   
                                                                         0.0010         0.550612   
           12              3         10        1          (256, 256)     0.0001         0.481475   
                                                                         0.0005         0.550604   
                                                                         0.0010         0.553193   
                                                          (512, 512)     0.0001         0.500370   
                                                                         0.0005         0.555877   
                                                                         0.0010         0.551182   

                                                                                      

In [13]:
best_swmhau_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.669504,0.560952,"[0.7845232313420755, 0.4986642920747996, 0.399...",0.557306,"[0.7874553136171596, 0.5243445692883895, 0.360...",0.568658,"[0.7816129032258065, 0.47538200339558573, 0.44...",None,0.668082,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.671165,0.558731,"[0.7872169584069376, 0.4967860422405877, 0.392...",0.558031,"[0.7838183562519987, 0.541, 0.3492753623188406]",0.565674,"[0.7906451612903226, 0.4592529711375212, 0.447...",None,0.671541,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.668051,0.556630,"[0.7848509266720387, 0.4884135472370766, 0.396...",0.554022,"[0.784219001610306, 0.5140712945590994, 0.3637...",0.562224,"[0.785483870967742, 0.46519524617996605, 0.435...",None,0.661792,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_umap_kfold_20["f1"].mean()

0.5587708818192851

In [15]:
best_swmhau_network_umap_kfold_20["precision"].mean()

0.5564533372275692

In [16]:
best_swmhau_network_umap_kfold_20["recall"].mean()

0.5655187367945068

In [17]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.78553037, 0.49462129, 0.39616098])

In [18]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.78516422, 0.52647195, 0.35772383])

In [19]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.78591398, 0.46661007, 0.44403216])

## Random Projections

In [ ]:
swmhau_network_grp_kfold_20, best_swmhau_network_grp_kfold_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

In [ ]:
swmhau_network_grp_kfold_20.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "dropout_rate",
    "learning_rate"
]).mean()

In [ ]:
best_swmhau_network_grp_kfold_20

In [ ]:
best_swmhau_network_grp_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

# w=35

In [ ]:
size = 35

## UMAP

In [ ]:
swmhau_network_umap_kfold_35, best_swmhau_network_umap_kfold_35, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

In [ ]:
swmhau_network_umap_kfold_35.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "learning_rate"
]).mean()

In [ ]:
best_swmhau_network_umap_kfold_35

In [ ]:
best_swmhau_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
swmhau_network_grp_kfold_35, best_swmhau_network_grp_kfold_35, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

In [ ]:
swmhau_network_grp_kfold_35.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "dropout_rate",
    "learning_rate"
]).mean()

In [ ]:
best_swmhau_network_grp_kfold_35

In [ ]:
best_swmhau_network_grp_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["recall_scores"]).mean(axis=0)